# **Binance Currency Pair Analysis**

In this notebook, we will conduct a general evaluation of the data provided by the Binance API, focusing on currency pairs that involve fiat currencies. The goal is to identify and analyze the available pairs, as well as to detect any new fiat currencies that may have been added to the platform.

## **1. Importing Libraries**
First, we will import the necessary libraries to perform our API requests and manipulate the data.

In [1]:
import requests
import pandas as pd
import os

# Lista completa de monedas fiat
fiat_currencies = [
    'USD', 'EUR', 'GBP', 'JPY', 'AUD', 'CAD', 'CHF', 'CNY', 'INR', 'RUB', 
    'BRL', 'ARS', 'NGN', 'TRY', 'MXN', 'ZAR', 'PLN', 'SEK', 'NOK', 'DKK', 
    'HKD', 'SGD', 'NZD', 'KRW', 'THB', 'MYR', 'IDR', 'PHP', 'VND', 'CZK', 
    'HUF', 'RON', 'BGN', 'HRK', 'RSD', 'UAH', 'KZT', 'AZN', 'GEL', 'BYN',
    'EGP', 'ILS', 'SAR', 'AED', 'QAR', 'KWD', 'BHD', 'OMR', 'TND', 'MAD', 
    'DZD', 'TWD', 'LKR', 'PKR', 'BDT', 'KES', 'TZS', 'UGX', 'GHS', 'XAF', 
    'XOF', 'XPF', 'MUR', 'SCR', 'MGA', 'ZMW', 'MWK', 'BWP', 'NAD', 'SLL',
    'LRD', 'LSL', 'SZL', 'MZN', 'BIF', 'CDF', 'DJF', 'ETB', 'GNF', 'HTG',
    'LRD', 'MGA', 'NPR', 'PGK', 'PGK', 'SHP', 'SLL', 'SOS', 'SSP', 'STD',
    'SYP', 'TOP', 'TTD', 'TWD', 'VUV', 'WST', 'XCD', 'YER', 'ZMW'
]

# Definir la carpeta base donde se guardarán los archivos CSV
base_folder = 'datos_historicos_binance'

# Crear la carpeta base si no existe
if not os.path.exists(base_folder):
    os.makedirs(base_folder)


## **2. Retrieving Information on Currency Pairs in Binance**
Here, we define a function to retrieve all available currency pairs on Binance using their API. This function extracts relevant information for each pair, including the base and quote currencies.

In [2]:
def obtener_info_pares_binance():
    # URL del endpoint de exchangeInfo de Binance
    url = 'https://api.binance.com/api/v3/exchangeInfo'
    
    # Realizar la solicitud GET al endpoint
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir los datos de respuesta a formato JSON
        data = response.json()
        
        # Crear un diccionario para almacenar la información sobre los pares de divisas
        info_pares = {}
        
        # Iterar sobre los datos para obtener la información relevante sobre cada par de divisas
        for symbol_info in data['symbols']:
            symbol = symbol_info['symbol']
            base_currency = symbol_info['baseAsset']
            quote_currency = symbol_info['quoteAsset']
            
            # Almacenar la información en el diccionario
            info_pares[symbol] = {'base_currency': base_currency, 'quote_currency': quote_currency}
        
        return info_pares
    else:
        print(f"Error al obtener información sobre los pares de divisas de Binance. Código de estado: {response.status_code}")
        return None

# Obtener información sobre los pares de divisas
info_pares = obtener_info_pares_binance()

## **3. Filtering Currency Pairs by Fiat Currencies**
Now that we have all the information on currency pairs in Binance, we will filter to obtain only those pairs that involve any of the fiat currencies we defined earlier. This will allow us to focus on the currency pairs that are relevant to our analysis.

In [3]:
# Filtrar para obtener solo los pares de interés con las monedas fiat
fiat_pairs = [pair for pair, currencies in info_pares.items() if currencies['quote_currency'] in fiat_currencies or currencies['base_currency'] in fiat_currencies]

print(f"Total pairs with fiat currencies: {len(fiat_pairs)}")


Total pairs with fiat currencies: 452


## **Conclusion**
In this initial stage, we have retrieved and filtered the Binance currency pairs that involve fiat currencies. This prepares us for more detailed and specific analyses in the following sections of this notebook.


# **Downloading and Saving Historical Data for Fiat-Crypto Pairs on Binance**

In this section, we will download historical daily data for the fiat-crypto pairs we've identified from Binance's API. The data will be saved in monthly CSV files, organized by fiat currency and cryptocurrency.


In [4]:
import requests
import pandas as pd
import os
from tqdm.notebook import tqdm

# Define the base folder where CSV files will be stored
base_folder = 'datos_historicos_binance'

# Create the base folder if it does not exist
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

# Initialize a counter and a list to track saved files
total_files_saved = 0
saved_files = []

def obtener_datos_y_guardar_csv(symbol, base_currency, quote_currency):
    global total_files_saved
    global saved_files

    # Build the URL for the Binance API
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d'
    
    # Send a GET request to the API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert the response data to JSON format
        data = response.json()
        
        # Create a pandas DataFrame with the data
        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume', 
            'close_time', 'quote_asset_volume', 'number_of_trades', 
            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
        ])
        
        # Convert the 'timestamp' column to datetime format
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        
        # Determine the fiat currency and crypto currency based on their position
        if base_currency in fiat_currencies:
            fiat_currency = base_currency
            crypto_currency = quote_currency
        else:
            fiat_currency = quote_currency
            crypto_currency = base_currency
        
        # Create a folder for the fiat currency if it doesn't exist
        fiat_folder_path = os.path.join(base_folder, fiat_currency)
        if not os.path.exists(fiat_folder_path):
            os.makedirs(fiat_folder_path)
        
        # Create a subfolder for the cryptocurrency if it doesn't exist
        crypto_folder_path = os.path.join(fiat_folder_path, crypto_currency)
        if not os.path.exists(crypto_folder_path):
            os.makedirs(crypto_folder_path)
        
        # Save the data into monthly CSV files with daily data inside
        for month, df_month in df.groupby(df['timestamp'].dt.to_period('M')):
            csv_file_path = os.path.join(crypto_folder_path, f'{fiat_currency.lower()}_{crypto_currency.lower()}_{month.start_time.strftime("%Y-%m")}.csv')
            df_month.to_csv(csv_file_path, index=False)
            total_files_saved += 1
            saved_files.append(csv_file_path)
    else:
        print(f"Error fetching data for {symbol}. Status code: {response.status_code}")

# Download data for all the pairs of interest with a progress bar
for pair in tqdm(fiat_pairs, desc="Processing pairs"):
    base_currency = info_pares[pair]['base_currency']
    quote_currency = info_pares[pair]['quote_currency']
    obtener_datos_y_guardar_csv(pair, base_currency, quote_currency)

# Summary message
print(f"Total files saved: {total_files_saved}")

Processing pairs:   0%|          | 0/452 [00:00<?, ?it/s]

Total files saved: 5793


In [5]:
import os
import pandas as pd
from tqdm.notebook import tqdm

# Initialize a list to store the DataFrames
dfs = []

# Get the list of fiat currencies to iterate over and initialize the progress bar
fiat_currencies = [f for f in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, f))]

# Iterate through the fiat currency folders with a progress bar
for fiat_currency in tqdm(fiat_currencies, desc="Processing fiat currencies"):
    fiat_folder_path = os.path.join(base_folder, fiat_currency)
    
    # Iterate through the crypto currency folders
    for crypto_currency in os.listdir(fiat_folder_path):
        crypto_folder_path = os.path.join(fiat_folder_path, crypto_currency)
        if os.path.isdir(crypto_folder_path):
            # Iterate through the CSV files
            for csv_file in os.listdir(crypto_folder_path):
                csv_file_path = os.path.join(crypto_folder_path, csv_file)
                if csv_file_path.endswith('.csv'):
                    # Create a pandas DataFrame from the CSV data
                    df = pd.read_csv(csv_file_path)
                    # Add columns for fiat and crypto currencies
                    df['fiat_currency'] = fiat_currency
                    df['crypto_currency'] = crypto_currency
                    # Append the DataFrame to the list
                    dfs.append(df)

# Combine all the DataFrames into one
final_df = pd.concat(dfs, ignore_index=True)

# Display the first few rows of the final DataFrame to check the result
print(final_df.head())


Processing fiat currencies:   0%|          | 0/15 [00:00<?, ?it/s]

    timestamp   open   high    low  close    volume     close_time  \
0  2024-05-01  17.42  17.50  17.28  17.38  421736.0  1714607999999   
1  2024-05-02  17.37  17.39  17.06  17.14  481475.0  1714694399999   
2  2024-05-03  17.15  17.45  17.12  17.25  399922.0  1714780799999   
3  2024-05-04  17.21  17.29  17.14  17.16  233538.0  1714867199999   
4  2024-05-05  17.22  17.25  17.14  17.18  166996.0  1714953599999   

   quote_asset_volume  number_of_trades  taker_buy_base_asset_volume  \
0          7338720.35              2687                     252933.0   
1          8276155.90              2111                     290720.0   
2          6883074.60              2454                     240621.0   
3          4016583.00              1716                     125347.0   
4          2871544.49              1473                      98079.0   

   taker_buy_quote_asset_volume  ignore fiat_currency crypto_currency  
0                    4405772.29       0           MXN            USDT  
1 

In [6]:
# Check if there are any missing values in the dataset
if final_df.isnull().sum().sum() == 0:
    print("Data check complete: No missing values. All data is clean and ready for analysis.")
else:
    print("Data check complete: There are missing values in the dataset. Further inspection is needed.")

# Basic descriptive statistics (can be optional or removed if not needed for output)
final_stats = final_df.describe()
print("Basic statistics generated. Data is ready for further analysis.")

Data check complete: No missing values. All data is clean and ready for analysis.
Basic statistics generated. Data is ready for further analysis.


In [7]:
import pandas as pd

# Ensure 'timestamp' is in datetime format
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Filter the data to include only the last month's data
last_month = final_df['timestamp'].max() - pd.DateOffset(months=1)
filtered_df = final_df[final_df['timestamp'] >= last_month]

# Get the list of all fiat currencies in the dataset
all_fiat_currencies = final_df['fiat_currency'].unique()
print(f"All fiat currencies in the dataset (total {len(all_fiat_currencies)}): {sorted(all_fiat_currencies)}")

# Get the list of fiat currencies present in the last month's data
fiat_currencies_last_month = filtered_df['fiat_currency'].unique()
print(f"Fiat currencies in the last month's data (total {len(fiat_currencies_last_month)}): {sorted(fiat_currencies_last_month)}")

# Identify fiat currencies that have BTC pairs
btc_fiat_currencies = filtered_df[filtered_df['crypto_currency'] == 'BTC']['fiat_currency'].unique()
print(f"Fiat currencies with BTC pairs (total {len(btc_fiat_currencies)}): {sorted(btc_fiat_currencies)}")

# Identify fiat currencies that have USDT pairs
usdt_fiat_currencies = filtered_df[filtered_df['crypto_currency'] == 'USDT']['fiat_currency'].unique()
print(f"Fiat currencies with USDT pairs (total {len(usdt_fiat_currencies)}): {sorted(usdt_fiat_currencies)}")

# Identify fiat currencies without BTC pairs
fiat_without_btc = set(fiat_currencies_last_month) - set(btc_fiat_currencies)
print(f"Fiat currencies without BTC pairs (total {len(fiat_without_btc)}): {sorted(fiat_without_btc)}")

# Identify fiat currencies without USDT pairs
fiat_without_usdt = set(fiat_currencies_last_month) - set(usdt_fiat_currencies)
print(f"Fiat currencies without USDT pairs (total {len(fiat_without_usdt)}): {sorted(fiat_without_usdt)}")

# Identify fiat currencies without either BTC or USDT pairs
fiat_without_btc_or_usdt = fiat_without_btc.union(fiat_without_usdt)
print(f"Fiat currencies without BTC or USDT pairs (total {len(fiat_without_btc_or_usdt)}): {sorted(fiat_without_btc_or_usdt)}")


All fiat currencies in the dataset (total 15): ['ARS', 'AUD', 'BRL', 'CZK', 'EUR', 'GBP', 'JPY', 'MXN', 'NGN', 'PLN', 'RON', 'RUB', 'TRY', 'UAH', 'ZAR']
Fiat currencies in the last month's data (total 11): ['ARS', 'BRL', 'CZK', 'EUR', 'JPY', 'MXN', 'PLN', 'RON', 'TRY', 'UAH', 'ZAR']
Fiat currencies with BTC pairs (total 10): ['ARS', 'BRL', 'EUR', 'JPY', 'MXN', 'PLN', 'RON', 'TRY', 'UAH', 'ZAR']
Fiat currencies with USDT pairs (total 10): ['ARS', 'BRL', 'CZK', 'EUR', 'MXN', 'PLN', 'RON', 'TRY', 'UAH', 'ZAR']
Fiat currencies without BTC pairs (total 1): ['CZK']
Fiat currencies without USDT pairs (total 1): ['JPY']
Fiat currencies without BTC or USDT pairs (total 2): ['CZK', 'JPY']


In [8]:
# Counting the number of unique cryptos per fiat currency
crypto_count_per_fiat = final_df.groupby('fiat_currency')['crypto_currency'].nunique()

# Display the count
print(crypto_count_per_fiat)

fiat_currency
ARS      3
AUD     30
BRL     46
CZK      1
EUR     63
GBP     25
JPY      7
MXN      3
NGN     10
PLN      4
RON      4
RUB     25
TRY    219
UAH      7
ZAR      5
Name: crypto_currency, dtype: int64


In [9]:
import pandas as pd
import plotly.graph_objects as go

# Ensure 'timestamp' is in datetime format
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Filter the data to include only the last month's data
last_month = final_df['timestamp'].max() - pd.DateOffset(months=1)
filtered_df = final_df[final_df['timestamp'] >= last_month]

# Drop duplicates to count each crypto-fiat pair only once
unique_pairs_df = filtered_df.drop_duplicates(subset=['crypto_currency', 'fiat_currency'])

# Count the frequency of each cryptocurrency in the filtered dataset
crypto_frequency = unique_pairs_df['crypto_currency'].value_counts()

# Create the Plotly bar chart
fig = go.Figure(data=[
    go.Bar(x=crypto_frequency.index, y=crypto_frequency.values, marker_color='skyblue')
])

# Customize the layout
fig.update_layout(
    title="Unique Frequency of Cryptocurrencies in the Last Month",
    xaxis_title="Cryptocurrency",
    yaxis_title="Unique Frequency",
    xaxis_tickangle=-45,
    template="plotly_white"
)

# Show the plot
fig.show()

# Simple confirmation message
print("Unique frequency plot of cryptocurrencies for the last month generated.")


Unique frequency plot of cryptocurrencies for the last month generated.


In [10]:
import pandas as pd
import plotly.graph_objects as go

# Ensure 'timestamp' is in datetime format
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Count the number of pairs for each cryptocurrency across the entire dataset
crypto_counts = final_df['crypto_currency'].value_counts()

# Identify the top 10 cryptocurrencies based on the entire dataset
top_10_cryptos = crypto_counts.nlargest(10).index.tolist()

# Filter the data to include only the last month's data
last_month = final_df['timestamp'].max() - pd.DateOffset(months=1)
filtered_df = final_df[final_df['timestamp'] >= last_month].copy()  # Use .copy() to avoid the SettingWithCopyWarning

# Group the remaining cryptocurrencies into "Others"
filtered_df.loc[:, 'crypto_category'] = filtered_df['crypto_currency'].apply(
    lambda x: x if x in top_10_cryptos else 'Others'
)

# Create a list of unique categories (top 10 cryptos + 'Others')
categories = top_10_cryptos + ['Others']

# List of all fiat currencies present in the entire dataset
all_fiats = sorted(final_df['fiat_currency'].unique())  # Sort fiat currencies alphabetically

# Initialize an empty matrix with zeros (no pairs) for all fiat currencies
matrix = pd.DataFrame(0, index=categories, columns=all_fiats)

# Fill the matrix with 1 where pairs exist
for _, row in filtered_df.iterrows():
    matrix.loc[row['crypto_category'], row['fiat_currency']] = 1

# Unified color scheme for all cryptos
colorscale = [[0, '#D3D3D3'], [1, '#4682B4']]  # Light gray for no pair, steel blue for all pairs

# Plot using Plotly
fig = go.Figure(data=go.Heatmap(
    z=matrix.values,
    x=matrix.columns,
    y=matrix.index,
    colorscale=colorscale,
    showscale=False,  # Hide color scale bar
))

# Update layout for better readability and custom Y-axis labels
fig.update_layout(
    title="Top 10 Crypto-Fiat Pair Presence and 'Others'",
    xaxis_title="Fiat Currency",
    yaxis_title="Cryptocurrency",
    yaxis=dict(
        tickmode='array',
        tickvals=list(range(len(categories))),
        ticktext=[f'<b>{crypto}</b>' if crypto == 'Others' else crypto for crypto in categories],
        tickfont=dict(size=10),
        side='left'
    ),
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(len(all_fiats))),
        ticktext=all_fiats  # Ensure all fiat currencies are shown on the X-axis
    ),
    template="plotly_white"
)

# Show the plot
fig.show()


In [11]:
import pandas as pd
import plotly.graph_objects as go
import warnings
from tqdm.notebook import tqdm  # To show progress bar

# Suppress the SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Ensure 'timestamp' is in datetime format
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Count the number of pairs for each cryptocurrency across the entire dataset
all_crypto_counts = final_df['crypto_currency'].value_counts()

# Identify the top 10 cryptocurrencies based on the entire dataset
top_10_cryptos = all_crypto_counts.nlargest(10).index.tolist()

# Define categories for Y-axis (top 10 cryptos + 'Others')
categories = top_10_cryptos + ['Others']

# Extract the month from the timestamp
final_df['month'] = final_df['timestamp'].dt.to_period('M')

# Extend the data to include up to 2024-08, even if there are no new data entries
all_months = pd.period_range(final_df['month'].min(), '2024-08', freq='M')

# List of all fiat currencies present in the dataset
all_fiats = sorted(final_df['fiat_currency'].unique())  # Sort fiat currencies alphabetically

# Initialize a list to store all frames
frames = []

# Generate a frame for each month
for month in tqdm(all_months, desc="Generating frames"):
    # Filter the data for the given month
    monthly_data = final_df[final_df['month'] == month].copy()

    # Group the remaining cryptocurrencies into "Others"
    if not monthly_data.empty:
        monthly_data.loc[:, 'crypto_category'] = monthly_data['crypto_currency'].apply(
            lambda x: x if x in top_10_cryptos else 'Others'
        )

    # Initialize an empty matrix with zeros (no pairs)
    matrix = pd.DataFrame(0, index=categories, columns=all_fiats)

    # Fill the matrix with 1 where pairs exist
    if not monthly_data.empty:
        for _, row in monthly_data.iterrows():
            matrix.loc[row['crypto_category'], row['fiat_currency']] = 1

    # Create a frame for this month
    frames.append(go.Frame(
        data=go.Heatmap(
            z=matrix.values,
            x=matrix.columns,
            y=matrix.index,
            colorscale=[[0, '#D3D3D3'], [1, '#4682B4']],  # Light gray for no pair, steel blue for all pairs
            showscale=False
        ),
        name=str(month)
    ))

# Plot using Plotly
fig = go.Figure(
    data=frames[0]['data'],  # Use the first month's data as the starting point
    layout=go.Layout(
        title="Top 10 Crypto-Fiat Pair Presence and 'Others' Over Time (All Fiat Currencies)",
        xaxis_title="Fiat Currency",
        yaxis_title="Cryptocurrency",
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(categories))),
            ticktext=[f'<b>{crypto}</b>' if crypto == 'Others' else crypto for crypto in categories],
            tickfont=dict(size=10),
            side='left'
        ),
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(len(all_fiats))),
            ticktext=all_fiats  # This is now sorted alphabetically
        ),
        template="plotly_white",
        updatemenus=[dict(type="buttons", showactive=False,
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, {"frame": {"duration": 500, "redraw": True},
                                                     "fromcurrent": True,
                                                     "transition": {"duration": 300}}]),
                                   dict(label="Pause",
                                        method="animate",
                                        args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                       "mode": "immediate",
                                                       "transition": {"duration": 0}}])])],
        sliders=[{
            'currentvalue': {
                'prefix': 'Month: ',
                'font': {'size': 20}
            },
            'pad': {'b': 10},
            'len': 0.9,
            'x': 0.1,
            'y': 0,
            'steps': [{
                'args': [[str(month)], {
                    'frame': {'duration': 500, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 300}
                }],
                'label': str(month),
                'method': 'animate'
            } for month in all_months]
        }]
    ),
    frames=frames  # Add the frames to the figure
)

# Show the plot
fig.show()


Generating frames:   0%|          | 0/59 [00:00<?, ?it/s]

In [12]:
fig.write_html("crypto_fiat_animation.html")

In [13]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species")
fig.show()


/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

